## Gaussian Discriminant Analysis

with varying covariance $\Sigma_1 \neq \Sigma_2 \neq \Sigma_3 ... \neq \Sigma_3$

In [3]:
import pandas as pd
import numpy as np

# To reset the printoptions
# np.set_printoptions(edgeitems=3,infstr='inf',linewidth=75, nanstr='nan', precision=8,suppress=False, threshold=1000, formatter=None)

np.set_printoptions(precision=4, suppress=True)

#np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

### For multi-class label y ={0, 1, 2, ...}

In [4]:
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
y = dataset.target
#y=(y>0).astype(int) 

target_names = list(dataset.target_names)
print(target_names)

['setosa', 'versicolor', 'virginica']


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112, 4), (38, 4), (112,), (38,))

In [18]:
class GaussianDiscAnalysis: 
    def compute_phi(self, y):
        n = len(y)
        phi = dict()
        for idx in range(len(self.classes)):
            phi[idx] = (1/n) * np.sum(y==idx)
        return phi
    
    def compute_mu(self, X, y):
        mu_dict = dict()
        for idx in range(len(self.classes)):
            # Add mu for each class
            mu_dict[idx] = np.sum(X[y==idx], axis=0)/ np.sum(y==idx)
        return mu_dict

    def compute_sigma(self, X, y):
        sigma_dict = dict()
        #n = len(X)
        #y = y.reshape(-1,1)
        Xmu = X.copy()
        for idx in range(len(self.classes)):
            n = np.sum(y==idx)
            Xmu_i = Xmu[y==idx] - self.mu[idx]
            sigma = (1/n) * Xmu_i.T@Xmu_i
            sigma_dict[idx] = sigma
        return sigma_dict
    
    
    def compute_Pxyi(self, X, idx):
        """Probability of X given y"""
        m = X.shape[1]
        sigma_inv = np.linalg.inv(self.sigma[idx])
        det_sigma = np.linalg.det(self.sigma[idx])
        #mu_i = mu(X, y, idx)
        Pxi = (1/((2*np.pi)**(m/2))) \
                *(1/(det_sigma**0.5)) \
                * np.exp(- 0.5*np.sum(((X-self.mu[idx])@sigma_inv)*(X-self.mu[idx]), axis=1))
    #     Pxi = np.log(1) \
    #             - np.log((2*np.pi)**(m/2)) \
    #             - np.log(np.sqrt(det_sigma)) \
    #             - np.sum(((X-mu_i)@sigma_inv)*(X-mu_i), axis=1)
        return Pxi
    
    def fit(self, X, y):
        """Computes mean, covariance and proabilities of y (phi)"""
        self.classes = np.unique(y)
        self.mu = self.compute_mu(X, y)
        self.sigma = self.compute_sigma(X, y)
        self.phi = self.compute_phi(y)
        
    def predict_proba(self, X):
        """Computes the probability of example belonging to that class"""
        n = len(X)
        Pyi = np.zeros((n, len(self.classes)))
        
        for idx in range(len(self.classes)):
            #print(self.compute_Pxyi(X, idx))
            py_i = self.compute_Pxyi(X, idx) * self.phi[idx]
            Pyi[:, idx] = py_i
        return Pyi
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)
    
    def generate_data(self, class_id, num_samples=1):
        """Generates new unseen dataset from a normal distribution
            given the mean of class and covariance
        """
        mean = self.mu[class_id]
        cov = self.sigma[class_id]
        return np.random.multivariate_normal(mean, cov, num_samples)

In [19]:
GDA = GaussianDiscAnalysis()

In [20]:
GDA.fit(X_train, y_train)

In [21]:
predictions = GDA.predict(X_test)

In [22]:
predictions

array([2, 2, 2, 1, 1, 2, 0, 0, 1, 1, 0, 2, 1, 2, 1, 0, 1, 0, 0, 2, 2, 2,
       1, 2, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 2, 1, 0, 1])

In [23]:
GDA.predict_proba(X_test)[0:5]

array([[0.    , 0.    , 0.1947],
       [0.    , 0.    , 0.1722],
       [0.    , 0.    , 0.2566],
       [0.    , 0.0373, 0.    ],
       [0.    , 0.0713, 0.0014]])

In [24]:
np.sum(predictions == y_test) / len(y_test)

1.0

### Generate new data

In [28]:
new_data = GDA.generate_data(class_id=1, num_samples=100)

In [29]:
new_data

array([[5.9675, 2.5222, 4.6476, 1.3642],
       [5.1838, 2.3462, 3.9423, 0.9555],
       [5.9963, 2.9849, 4.738 , 1.3645],
       [5.5439, 2.4985, 3.7781, 1.1449],
       [5.4531, 3.1839, 3.8739, 1.4079],
       [5.8617, 2.9757, 4.0662, 1.2893],
       [5.5628, 2.7596, 4.7163, 1.435 ],
       [5.7966, 3.1488, 4.6963, 1.6949],
       [5.6333, 2.4279, 4.3005, 1.2556],
       [5.6835, 2.2771, 4.1266, 1.2477],
       [6.151 , 2.8524, 4.3203, 1.3174],
       [5.8238, 2.7191, 4.3683, 1.3531],
       [5.4265, 2.3532, 3.6663, 1.1042],
       [5.7894, 2.859 , 4.1452, 1.3298],
       [5.5581, 2.8484, 3.9068, 1.5076],
       [6.3435, 3.1905, 4.535 , 1.4166],
       [6.1199, 2.3312, 4.5212, 1.2021],
       [6.4661, 2.897 , 5.3044, 1.5201],
       [5.8467, 3.1927, 4.2493, 1.5801],
       [5.8982, 2.6503, 5.045 , 1.4258],
       [6.01  , 2.9193, 4.4122, 1.4452],
       [6.3367, 2.8945, 5.0821, 1.8343],
       [6.5146, 3.3664, 4.6055, 1.6125],
       [5.1452, 2.1495, 3.3473, 1.1171],
       [5.5162, 

In [31]:
GDA.predict(new_data)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1])